# Importando e tratando as bases de dados

><h1>Importação dos dados

>><h2>Chuva_2019

In [1]:
import requests
from io import StringIO

# parameters for the request
owner = 'danplevs'
token = '10972548eeae219489116329ded0f3850ddeec5a'
repo = 'geohmount'
path = 'geohmount-plotly/sazonalidade-chuva/chuva_2019.csv'

# sending the request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

#convert string to StringIO object
data = StringIO(r.text)

In [2]:
import pandas as pd

df = pd.read_csv(data, sep=';')
df

,estacao,chuva,sd
0,Média Anual - BM\n(2007 - 2019),1262.7,229.7
1,BM \n(E. Pedro do Rio),1189.3,NaN
2,BM \n(E. Bonfim),1195.8,NaN
3,Média Anual - SB\n(2007 - 2019),2837.6,467.8
4,SB \n(E. Parnaso),3055.4,NaN
5,Média Anual - SM\n(2007 - 2019),1317.9,278.5
6,SM \n(E. Santa Marta),1590.1,NaN
7,SM \n(E. Caparaó),1171.5,NaN


>><h2>Sazonalidade

In [3]:
import requests
from io import StringIO

# parameters for the request
owner = 'danplevs'
token = '10972548eeae219489116329ded0f3850ddeec5a'
repo = 'geohmount'
path = 'geohmount-plotly/sazonalidade-chuva/sazonalidade.csv'

# sending the request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

#convert string to StringIO object
data = StringIO(r.text)

In [4]:
import pandas as pd

df2 = pd.read_csv(data, sep=';')
df2.head()

,mês,SM (Estação Caparaó),Série Histórica (2007-2019) e DP,sd_SM,BM (Estação Pedro do Rio),Série Histórica (2007-2019) e DP.1,sd_BM,SB (Estação PARNASO),Série Histórica (2007-2019) e DP.2,sd_SB
0,fev/20,336.0,119.91,58.82,316.8,140.16,66.45,299.2,202.35,118.87
1,jan/20,514.4,177.99,140.34,269.0,260.84,85.93,590.8,411.87,186.21
2,dez/19,235.8,283.67,122.68,248.7,202.29,87.54,335.4,398.80,146.73
3,nov/19,319.0,225.65,95.26,106.5,184.39,54.65,358.6,410.75,114.02
4,out/19,57.6,96.50,41.64,109.6,96.72,47.40,142.8,230.74,106.18


In [5]:
# invertendo o dataframe
df2 = df2.reindex(index=df2.index[::-1])
df2.head()

,mês,SM (Estação Caparaó),Série Histórica (2007-2019) e DP,sd_SM,BM (Estação Pedro do Rio),Série Histórica (2007-2019) e DP.1,sd_BM,SB (Estação PARNASO),Série Histórica (2007-2019) e DP.2,sd_SB
25,jan/18,147.2,177.99,140.34,255.9,260.84,130.54,491.8,411.87,186.21
24,fev/18,210.1,119.91,58.82,166.7,140.16,66.45,299.2,202.35,118.87
23,mar/18,406.3,185.92,120.95,243.6,179.23,63.64,533.4,371.11,147.03
22,abr/18,155.6,96.65,38.76,86.5,62.93,30.26,218.0,244.45,100.81
21,mai/18,35.7,45.13,35.84,40.6,38.41,19.32,241.0,150.19,69.94


In [6]:
# renomeando as séries históricas
df2.rename(columns={'Série Histórica (2007-2019) e DP': 'sh_SM', 'Série Histórica (2007-2019) e DP.1': 'sh_BM', 'Série Histórica (2007-2019) e DP.2': 'sh_SB'}, inplace=True)
df2.head()

,mês,SM (Estação Caparaó),sh_SM,sd_SM,BM (Estação Pedro do Rio),sh_BM,sd_BM,SB (Estação PARNASO),sh_SB,sd_SB
25,jan/18,147.2,177.99,140.34,255.9,260.84,130.54,491.8,411.87,186.21
24,fev/18,210.1,119.91,58.82,166.7,140.16,66.45,299.2,202.35,118.87
23,mar/18,406.3,185.92,120.95,243.6,179.23,63.64,533.4,371.11,147.03
22,abr/18,155.6,96.65,38.76,86.5,62.93,30.26,218.0,244.45,100.81
21,mai/18,35.7,45.13,35.84,40.6,38.41,19.32,241.0,150.19,69.94


# Criando o 1º gráfico

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(name='Bonfim',x=df.estacao[:3], y=df.chuva[:3], marker_color='rgb(0, 153, 216)', error_y=dict(type='data', array=df.sd[:3])
))
fig.add_trace(go.Bar(name='Soberbo',x=df.estacao[3:5], y=df.chuva[3:5], marker_color='rgb(0, 90, 74)', error_y=dict(type='data', array=df.sd[3:5])
))
fig.add_trace(go.Bar(name='Santa Marta',x=df.estacao[5:8], y=df.chuva[5:8], marker_color='rgb(86, 34, 18)', error_y=dict(type='data', array=df.sd[5:8])
))
fig.update_layout(font_family='Open Sans',  xaxis_tickfont_size=14, yaxis=dict(title='<b>Precipitação no ano de 2019 (mm)', titlefont_size=16, tickfont_size=14
))
fig.show()

# Criando o 2º gráfico

In [14]:
import plotly.graph_objects as go

blue = 'rgb(0, 153, 216)'
green = 'rgb(0, 90, 74)'
brown = 'rgb(86, 34, 18)'
fig2 = go.Figure()

fig2.add_trace(go.Bar(name='BM (Estação Pedro do Rio)', x=df2['mês'], y=df2['BM (Estação Pedro do Rio)'], marker=dict(color=blue)
))

fig2.add_trace(go.Scatter(name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_BM'], error_y=dict(color='black', type='data', array=df2['sd_BM'] ),line=dict(color=blue, width=3), visible='legendonly'
))

fig2.add_trace(go.Bar(name='SB (Estação PARNASO)',x=df2['mês'], y=df2['SB (Estação PARNASO)'], marker=dict(color=green)
))

fig2.add_trace(go.Scatter(name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_SB'], error_y=dict(color='black', type='data', array=df2['sd_SB']),line=dict(color=green, width=3), visible='legendonly'
))

fig2.add_trace(go.Bar(name='SM (Estação Caparaó)',x=df2['mês'], y=df2['SM (Estação Caparaó)'], marker=dict(color=brown)
))

fig2.add_trace(go.Scatter(name='Série Histórica (2007-2019)', x=df2['mês'], y=df2['sh_SM'], error_y=dict(color='black', type='data', array=df2['sd_SM']),line=dict(color=brown, width=3), visible='legendonly'
))

fig2.update_layout(barmode='group', xaxis_tickangle=-45, font_family='Open Sans', xaxis_tickfont_size=14, yaxis=dict(title='<b>Precipitação (mm)', titlefont_size=16, tickfont_size=14),
legend=dict(orientation='h', xanchor='center', yanchor='bottom', x=0.5, y=1.05
))

fig2.show()